# Imports necessários

## Bibliotecas

In [199]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
from permetrics.regression import RegressionMetric
import matplotlib.pyplot as plt 

import statistics as st
import sys
import math
import pandas as pd
import numpy as np
sys.path.insert(1, '..\Modelo')

from sklearn.model_selection import ParameterGrid
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from lssvr import LSSVR
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from eTS import eTS
from Simpl_eTS import Simpl_eTS
from exTS import exTS
from ePL import ePL
from eMG import eMG
from ePL_plus import ePL_plus
from ePL_KRLS_DISCO import ePL_KRLS_DISCO
from NMFIS import NMFIS
from NTSK import NTSK
from GEN_NMFIS import GEN_NMFIS
from GEN_NTSK import GEN_NTSK

# Neural Network
# Model
from keras.models import Sequential, Model
# Layers
from keras.layers import Input, InputLayer, Dense, Dropout, Conv1D, GRU, LSTM, MaxPooling1D, Flatten, SimpleRNN
#from tensorflow.keras.layers import Input
from tcn import TCN
# Optimizers
from keras.optimizers import SGD, Adam
# Save network
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

## Importando a série

In [47]:
df = pd.read_csv("../Dados/Dados refinados/timeseries.csv", index_col= 0)
df.head(10)

,mes,hora_sin,hora_cos,dia_semana_sin,dia_semana_cos,QTD_VEICULOS_USINA,primeira_quinzena,feriado,fim_semana,lag_1h,lag_1d,lag_7d,TPV
0,1,9.659258e-01,-0.258819,-0.974928,-0.222521,21,1,0,1,8.840000,6.538125,2.220000,8.484063
1,1,8.660254e-01,-0.500000,-0.974928,-0.222521,23,1,0,1,8.885333,6.293235,2.220000,8.150294
2,1,8.660254e-01,-0.500000,-0.974928,-0.222521,28,1,0,1,8.484063,6.093056,2.476667,7.830648
3,1,7.071068e-01,-0.707107,-0.974928,-0.222521,35,1,0,1,8.150294,5.945197,2.605000,7.581667
4,1,7.071068e-01,-0.707107,-0.974928,-0.222521,32,1,0,1,7.830648,5.780604,2.682000,7.362583
5,1,5.000000e-01,-0.866025,-0.974928,-0.222521,29,1,0,1,7.581667,5.620099,2.733333,7.186905
6,1,5.000000e-01,-0.866025,-0.974928,-0.222521,29,1,0,1,7.362583,5.523352,2.770000,6.978712
7,1,2.588190e-01,-0.965926,-0.974928,-0.222521,30,1,0,1,7.186905,5.412337,2.797500,6.898043
8,1,2.588190e-01,-0.965926,-0.974928,-0.222521,32,1,0,1,6.978712,5.309656,2.818889,6.772153
9,1,1.224647e-16,-1.000000,-0.974928,-0.222521,31,1,0,1,6.898043,5.209537,2.808000,6.632307


# Transformação de dados

## Normalizando os dados

In [48]:
#Separando X e y

X = df.iloc[:, :-1]
y = df.iloc[:,-1]

In [49]:
#Definindo o scalers
scaler_X, scaler_y = MinMaxScaler(),MinMaxScaler()

#fit
scaler_X.fit(X[['mes','QTD_VEICULOS_USINA']])
scaler_y.fit(pd.DataFrame(y))

#transform

X[['mes','QTD_VEICULOS_USINA']] = scaler_X.transform(X[['mes','QTD_VEICULOS_USINA']])
y = scaler_y.transform(pd.DataFrame(y))

In [50]:
cat_cols = ['mes', 'primeira_quinzena', 'feriado', 'fim_semana']

for col in cat_cols:
    X[col] = X[col].astype('category')

## Separação train, val, test

In [134]:
X.shape[0]

10271

In [135]:
# Separando os índices do treino, validacao e teste

index_train = round(X.shape[0]*0.5)
index_val = round(X.shape[0]*0.3) + index_train

#Separando as bases
X_train, y_train = X.values[:index_train], y[:index_train]
X_val, y_val = X.values[index_train:index_val], y[index_train:index_val]
X_test, y_test =  X.values[index_val:], y[index_val:]


y_test = scaler_y.inverse_transform(y_test)

# Modelagem


## Modelos Clássicos

In [168]:
# Summary
columns = ["Model_Name", "NRMSE", "NDEI", "MAPE", "Rules", "Best_Params"]
results = pd.DataFrame(columns = columns)

# Predictions
predictions = pd.DataFrame(y_test, columns= ['y_test'])

### KNN

In [169]:
Model_Name = "KNN"

# Define Grid Search parameters
parameters = {'n_neighbors': [2, 3, 4, 5, 10, 20]}
grid = ParameterGrid(parameters)

lower_rmse = np.inf
for i,param in enumerate(grid):
    
    print(f"[{i+1}/{len(grid)}] {param}" )
  

    # Optimize parameters
    KNN = KNeighborsRegressor(**param)
    KNN.fit(X_train,y_train)
    # Make predictions
    y_pred = KNN.predict(X_val)
    
    
    # Calculating the error metrics
    # Compute the Root Mean Square Error
    RMSE = math.sqrt(mean_squared_error(y_val, y_pred))
    
    if RMSE < lower_rmse:
        lower_rmse = RMSE
        best_KNN_params = param

# Optimized parameters

new_X_train, new_y_train = X.values[:index_val], y[:index_val]

KNN = KNeighborsRegressor(**best_KNN_params)
KNN.fit(new_X_train, new_y_train)

# Make predictions
y_pred = KNN.predict(X_test)

# Save predictions to dataframe
y_pred = scaler_y.inverse_transform(y_pred)
predictions[f'{Model_Name}'] = y_pred

# Model name
print(f'\nResults for {Model_Name}: \n')

# Calculating the error metrics

# Compute the Root Mean Square Error
RMSE = math.sqrt(mean_squared_error(y_test, y_pred))
print("\nRMSE:", RMSE)
# Compute the Normalized Root Mean Square Error
NRMSE = RegressionMetric(y_test, y_pred).normalized_root_mean_square_error()
print("NRMSE:", NRMSE)
# Compute the Non-Dimensional Error Index
NDEI= RMSE/st.stdev(np.asfarray(y_test.flatten()))
print("NDEI:", NDEI)
# Compute the Mean Absolute Error
MAE = mean_absolute_error(y_test, y_pred)
print("MAE:", MAE)
# Compute the Mean Absolute Percentage Error
MAPE = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE:", MAPE)
# Number of rules
Rules = "-"
print("Rules:", Rules)


print(f'\nBest params: \n{best_KNN_params}')

# Store results to dataframe
newrow = pd.DataFrame([[Model_Name, NRMSE, NDEI, MAPE, Rules, best_KNN_params]], columns=columns)
results = pd.concat([results, newrow], ignore_index=True)

# salvando em planilha
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")

results.to_excel(f"../Relatórios/results_{timestamp}.xlsx")
predictions.to_excel(f"../Relatórios/predictions_{timestamp}.xlsx")

[1/6] {'n_neighbors': 2}
[2/6] {'n_neighbors': 3}
[3/6] {'n_neighbors': 4}
[4/6] {'n_neighbors': 5}
[5/6] {'n_neighbors': 10}
[6/6] {'n_neighbors': 20}

Results for KNN: 


RMSE: 1.5303760205592174
NRMSE: 0.5508163043897
NDEI: 0.5430557275858096
MAE: 0.8654752944122002
MAPE: 0.44578320258869514
Rules: -

Best params: 
{'n_neighbors': 5}


C:\Users\thevh\AppData\Local\Temp\ipykernel_1508\92911531.py:71: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, newrow], ignore_index=True)


### Regression Tree

In [170]:
Model_Name = "Regression Tree"

# Define Grid Search parameters
parameters = {'max_depth': [2, 4, 6, 8, 10, 20, 30, 50, 100]}
grid = ParameterGrid(parameters)

lower_rmse = np.inf
for i,param in enumerate(grid):
    
    print(f"[{i+1}/{len(grid)}] {param}" )

    # Optimize parameters
    RT = DecisionTreeRegressor(**param)
    RT.fit(X_train,y_train)
    # Make predictions
    y_pred = RT.predict(X_val)
    
    
    # Calculating the error metrics
    # Compute the Root Mean Square Error
    RMSE = math.sqrt(mean_squared_error(y_val, y_pred))
    
    if RMSE < lower_rmse:
        lower_rmse = RMSE
        best_RT_params = param

# Optimized parameters
new_X_train, new_y_train = X.values[:index_val], y[:index_val]

RT = DecisionTreeRegressor(**best_RT_params)
RT.fit(new_X_train,new_y_train)
# Make predictions
y_pred = RT.predict(X_test)

# Save predictions to dataframe
y_pred = scaler_y.inverse_transform(y_pred.reshape(1, -1)).reshape(-1,1)
predictions[f'{Model_Name}'] = y_pred


# Model name
print(f'\nResults for {Model_Name}: \n')

# Calculating the error metrics
# Compute the Root Mean Square Error
RMSE = math.sqrt(mean_squared_error(y_test, y_pred))
print("\nRMSE:", RMSE)
# Compute the Normalized Root Mean Square Error
NRMSE = RegressionMetric(y_test, y_pred).normalized_root_mean_square_error()
print("NRMSE:", NRMSE)
# Compute the Non-Dimensional Error Index
NDEI= RMSE/st.stdev(np.asfarray(y_test.flatten()))
print("NDEI:", NDEI)
# Compute the Mean Absolute Error
MAE = mean_absolute_error(y_test, y_pred)
print("MAE:", MAE)
# Compute the Mean Absolute Percentage Error
MAPE = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE:", MAPE)
# Number of rules
Rules = "-"
print("Rules:", Rules)

print(f'\nBest params: \n{best_RT_params}')
   
RT_ = f'{Model_Name} & {NRMSE:.2f} & {NDEI:.2f} & {MAPE:.2f} & {Rules}'

# Store results to dataframe
newrow = pd.DataFrame([[Model_Name, NRMSE, NDEI, MAPE, Rules, best_RT_params]], columns=columns)
results = pd.concat([results, newrow], ignore_index=True)

# salvando em planilha
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")

results.to_excel(f"../Relatórios/results_{timestamp}.xlsx")
predictions.to_excel(f"../Relatórios/predictions_{timestamp}.xlsx")

[1/9] {'max_depth': 2}
[2/9] {'max_depth': 4}
[3/9] {'max_depth': 6}
[4/9] {'max_depth': 8}
[5/9] {'max_depth': 10}
[6/9] {'max_depth': 20}
[7/9] {'max_depth': 30}
[8/9] {'max_depth': 50}
[9/9] {'max_depth': 100}

Results for Regression Tree: 


RMSE: 0.8452041879227032
NRMSE: 0.2926542016926031
NDEI: 0.2999216983700617
MAE: 0.3504340154219335
MAPE: 0.11005237502562898
Rules: -

Best params: 
{'max_depth': 8}


### SVM

In [171]:
Model_Name = "SVM"

# Define Grid Search parameters
parameters = {'kernel': ['linear', 'rbf', 'sigmoid'], 'C':[0.01,0.1,1,10], 'gamma': [0.01,0.5,1,10,50]}

grid = ParameterGrid(parameters)

lower_rmse = np.inf
for i,param in enumerate(grid):
    
    print(f"[{i+1}/{len(grid)}] {param}" )
    
    # Optimize parameters
    SVM = SVR(**param)
    SVM.fit(X_train,y_train.ravel())
    # Make predictions
    y_pred = SVM.predict(X_val)
    
    
    # Calculating the error metrics
    # Compute the Root Mean Square Error
    RMSE = math.sqrt(mean_squared_error(y_val, y_pred))
    
    if RMSE < lower_rmse:
        lower_rmse = RMSE
        best_SVM_params = param

# # Use best parameters1 to save time looking for best parameters2
# parameters2 = {'kernel': ['poly'], 'C':[best_SVM_params['C']], 'gamma': [best_SVM_params['gamma']], 'degree': [2,3]}

# grid2 = ParameterGrid(parameters2)

# for param in grid2:
    
#     #print(param)
    
#     # Optimize parameters
#     SVM = SVR(**param)
#     SVM.fit(X_train,y_train)
#     # Make predictions
#     y_pred = SVM.predict(X_val)
#     
    
#     # Calculating the error metrics
#     # Compute the Root Mean Square Error
#     RMSE = math.sqrt(mean_squared_error(y_val, y_pred))
    
#     if RMSE < lower_rmse:
#         lower_rmse = RMSE
#         best_SVM_params = param


# Optimized parameters

new_X_train, new_y_train = X.values[:index_val], y[:index_val]

SVM = SVR(**best_SVM_params)
SVM.fit(new_X_train,new_y_train.ravel())
# Make predictions
y_pred = SVM.predict(X_test)

# Save predictions to dataframe
y_pred = scaler_y.inverse_transform(y_pred.reshape(1, -1)).reshape(-1,1)
predictions[f'{Model_Name}'] = y_pred

# Model name
print(f'\nResults for {Model_Name}: \n')

# Calculating the error metrics
# Compute the Root Mean Square Error

RMSE = math.sqrt(mean_squared_error(y_test, y_pred))
print("\nRMSE:", RMSE)
# Compute the Normalized Root Mean Square Error
NRMSE = RegressionMetric(y_test, y_pred).normalized_root_mean_square_error()
print("NRMSE:", NRMSE)
# Compute the Non-Dimensional Error Index
NDEI= RMSE/st.stdev(np.asfarray(y_test.flatten()))
print("NDEI:", NDEI)
# Compute the Mean Absolute Error
MAE = mean_absolute_error(y_test, y_pred)
print("MAE:", MAE)
# Compute the Mean Absolute Percentage Error
MAPE = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE:", MAPE)
# Number of rules
Rules = "-"
print("Rules:", Rules)

SVM_ = f'{Model_Name} & {NRMSE:.2f} & {NDEI:.2f} & {MAPE:.2f} & {Rules}'

print(f'\n{best_SVM_params}')
   
# Store results to dataframe
newrow = pd.DataFrame([[Model_Name, NRMSE, NDEI, MAPE, Rules, best_SVM_params]], columns=columns)
results = pd.concat([results, newrow], ignore_index=True)

# salvando em planilha
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")

results.to_excel(f"../Relatórios/results_{timestamp}.xlsx")
predictions.to_excel(f"../Relatórios/predictions_{timestamp}.xlsx")

[1/60] {'C': 0.01, 'gamma': 0.01, 'kernel': 'linear'}
[2/60] {'C': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}
[3/60] {'C': 0.01, 'gamma': 0.01, 'kernel': 'sigmoid'}
[4/60] {'C': 0.01, 'gamma': 0.5, 'kernel': 'linear'}
[5/60] {'C': 0.01, 'gamma': 0.5, 'kernel': 'rbf'}
[6/60] {'C': 0.01, 'gamma': 0.5, 'kernel': 'sigmoid'}
[7/60] {'C': 0.01, 'gamma': 1, 'kernel': 'linear'}
[8/60] {'C': 0.01, 'gamma': 1, 'kernel': 'rbf'}
[9/60] {'C': 0.01, 'gamma': 1, 'kernel': 'sigmoid'}
[10/60] {'C': 0.01, 'gamma': 10, 'kernel': 'linear'}
[11/60] {'C': 0.01, 'gamma': 10, 'kernel': 'rbf'}
[12/60] {'C': 0.01, 'gamma': 10, 'kernel': 'sigmoid'}
[13/60] {'C': 0.01, 'gamma': 50, 'kernel': 'linear'}
[14/60] {'C': 0.01, 'gamma': 50, 'kernel': 'rbf'}
[15/60] {'C': 0.01, 'gamma': 50, 'kernel': 'sigmoid'}
[16/60] {'C': 0.1, 'gamma': 0.01, 'kernel': 'linear'}
[17/60] {'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}
[18/60] {'C': 0.1, 'gamma': 0.01, 'kernel': 'sigmoid'}
[19/60] {'C': 0.1, 'gamma': 0.5, 'kernel': 'linear'}
[20/6

### Gradient Boosting Model

In [180]:
Model_Name = "GB"

# Define Grid Search parameters
parameters = {'learning_rate':[0.01, 0.05, 0.1, 0.5, 0.9], 'n_estimators': [2, 4, 8, 16, 32, 64, 100, 200]}

grid = ParameterGrid(parameters)

lower_rmse = np.inf
for i,param in enumerate(grid):
    
    print(f"[{i+1}/{len(grid)}] {param}" )

    # Optimize parameters
    GB = GradientBoostingRegressor(**param)
    GB.fit(X_train,y_train.ravel())
    # Make predictions
    y_pred = GB.predict(X_val)
    
    
    # Calculating the error metrics
    # Compute the Root Mean Square Error
    RMSE = math.sqrt(mean_squared_error(y_val, y_pred))
    
    if RMSE < lower_rmse:
        lower_rmse = RMSE
        best_GB_params = param


# Optimized parameters

new_X_train, new_y_train = X.values[:index_val], y[:index_val]
GB = GradientBoostingRegressor(**best_GB_params)
GB.fit(new_X_train,new_y_train)
# Make predictions
y_pred = GB.predict(X_test)

# Save predictions to dataframe
y_pred = scaler_y.inverse_transform(y_pred.reshape(1, -1)).reshape(-1,1)
predictions[f'{Model_Name}'] = y_pred

# Model name
print(f'\nResults for {Model_Name}: \n')

# Calculating the error metrics
# Compute the Root Mean Square Error
RMSE = math.sqrt(mean_squared_error(y_test, y_pred))
print("\nRMSE:", RMSE)
# Compute the Normalized Root Mean Square Error
NRMSE = RegressionMetric(y_test, y_pred).normalized_root_mean_square_error()
print("NRMSE:", NRMSE)
# Compute the Non-Dimensional Error Index
NDEI= RMSE/st.stdev(np.asfarray(y_test.flatten()))
print("NDEI:", NDEI)
# Compute the Mean Absolute Error
MAE = mean_absolute_error(y_test, y_pred)
print("MAE:", MAE)
# Compute the Mean Absolute Percentage Error
MAPE = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE:", MAPE)
# Number of rules
Rules = "-"
print("Rules:", Rules)

GB_ = f'{Model_Name} & {NRMSE:.2f} & {NDEI:.2f} & {MAPE:.2f} & {Rules}'

print(f'\n{best_GB_params}')
   
# Store results to dataframe
newrow = pd.DataFrame([[Model_Name, NRMSE, NDEI, MAPE, Rules, best_GB_params]], columns=columns)
results = pd.concat([results, newrow], ignore_index=True)

# salvando em planilha
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")

results.to_excel(f"../Relatórios/results_{timestamp}.xlsx")
predictions.to_excel(f"../Relatórios/predictions_{timestamp}.xlsx")

[1/40] {'learning_rate': 0.01, 'n_estimators': 2}
[2/40] {'learning_rate': 0.01, 'n_estimators': 4}
[3/40] {'learning_rate': 0.01, 'n_estimators': 8}
[4/40] {'learning_rate': 0.01, 'n_estimators': 16}
[5/40] {'learning_rate': 0.01, 'n_estimators': 32}
[6/40] {'learning_rate': 0.01, 'n_estimators': 64}
[7/40] {'learning_rate': 0.01, 'n_estimators': 100}
[8/40] {'learning_rate': 0.01, 'n_estimators': 200}
[9/40] {'learning_rate': 0.05, 'n_estimators': 2}
[10/40] {'learning_rate': 0.05, 'n_estimators': 4}
[11/40] {'learning_rate': 0.05, 'n_estimators': 8}
[12/40] {'learning_rate': 0.05, 'n_estimators': 16}
[13/40] {'learning_rate': 0.05, 'n_estimators': 32}
[14/40] {'learning_rate': 0.05, 'n_estimators': 64}
[15/40] {'learning_rate': 0.05, 'n_estimators': 100}
[16/40] {'learning_rate': 0.05, 'n_estimators': 200}
[17/40] {'learning_rate': 0.1, 'n_estimators': 2}
[18/40] {'learning_rate': 0.1, 'n_estimators': 4}
[19/40] {'learning_rate': 0.1, 'n_estimators': 8}
[20/40] {'learning_rate': 0.1

C:\Users\thevh\AppData\Roaming\Python\Python39\site-packages\sklearn\ensemble\_gb.py:668: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?



Results for GB: 


RMSE: 1.0001489078029921
NRMSE: 0.41585961612248584
NDEI: 0.35490401412761174
MAE: 0.6997245278552686
MAPE: 0.6655793502657171
Rules: -

{'learning_rate': 0.1, 'n_estimators': 64}


### XGBoost

In [182]:
Model_Name = "XGBoost"

# Define Grid Search parameters
parameters = {'n_estimators':[250, 500], 'min_child_weight':[2,5], 
              'gamma':[i/10.0 for i in range(3,6)], 'max_depth': [2, 3, 4, 6, 7], 
              'eval_metric':['rmse'],'eta':[i/10.0 for i in range(3,6)]}

grid = ParameterGrid(parameters)

lower_rmse = np.inf
for i,param in enumerate(grid):
    
    print(f"[{i+1}/{len(grid)}] {param}" )


    # Optimize parameters
    XGBoost = XGBRegressor(**param)
    XGBoost.fit(X_train,y_train)
    # Make predictions
    y_pred = XGBoost.predict(X_val)
    
    
    # Calculating the error metrics
    # Compute the Root Mean Square Error
    RMSE = math.sqrt(mean_squared_error(y_val, y_pred))
    
    if RMSE < lower_rmse:
        lower_rmse = RMSE
        best_XGBoost_params = param

# Optimized parameters

new_X_train, new_y_train = X.values[:index_val], y[:index_val]
XGBoost = XGBRegressor(**best_XGBoost_params)
XGBoost.fit(new_X_train,new_y_train)
# Make predictions
y_pred = XGBoost.predict(X_test)

# Save predictions to dataframe
y_pred = scaler_y.inverse_transform(y_pred.reshape(1, -1)).reshape(-1,1)
predictions[f'{Model_Name}'] = y_pred

# Model name
print(f'\nResults for {Model_Name}: \n')

# Calculating the error metrics
# Compute the Root Mean Square Error
RMSE = math.sqrt(mean_squared_error(y_test, y_pred))
print("\nRMSE:", RMSE)
# Compute the Normalized Root Mean Square Error
NRMSE = RegressionMetric(y_test, y_pred).normalized_root_mean_square_error()
print("NRMSE:", NRMSE)
# Compute the Non-Dimensional Error Index
NDEI= RMSE/st.stdev(np.asfarray(y_test.flatten()))
print("NDEI:", NDEI)
# Compute the Mean Absolute Error
MAE = mean_absolute_error(y_test, y_pred)
print("MAE:", MAE)
# Compute the Mean Absolute Percentage Error
MAPE = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE:", MAPE)
# Number of rules
Rules = "-"
print("Rules:", Rules)
   
XGBoost_ = f'{Model_Name} & {NRMSE:.2f} & {NDEI:.2f} & {MAPE:.2f} & {Rules}'

print(f'\n{best_XGBoost_params}')

# Store results to dataframe
newrow = pd.DataFrame([[Model_Name, NRMSE, NDEI, MAPE, Rules, best_XGBoost_params]], columns=columns)
results = pd.concat([results, newrow], ignore_index=True)

# salvando em planilha
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")

results.to_excel(f"../Relatórios/results_{timestamp}.xlsx")
predictions.to_excel(f"../Relatórios/predictions_{timestamp}.xlsx")

[1/180] {'eta': 0.3, 'eval_metric': 'rmse', 'gamma': 0.3, 'max_depth': 2, 'min_child_weight': 2, 'n_estimators': 250}
[2/180] {'eta': 0.3, 'eval_metric': 'rmse', 'gamma': 0.3, 'max_depth': 2, 'min_child_weight': 2, 'n_estimators': 500}
[3/180] {'eta': 0.3, 'eval_metric': 'rmse', 'gamma': 0.3, 'max_depth': 2, 'min_child_weight': 5, 'n_estimators': 250}
[4/180] {'eta': 0.3, 'eval_metric': 'rmse', 'gamma': 0.3, 'max_depth': 2, 'min_child_weight': 5, 'n_estimators': 500}
[5/180] {'eta': 0.3, 'eval_metric': 'rmse', 'gamma': 0.3, 'max_depth': 3, 'min_child_weight': 2, 'n_estimators': 250}
[6/180] {'eta': 0.3, 'eval_metric': 'rmse', 'gamma': 0.3, 'max_depth': 3, 'min_child_weight': 2, 'n_estimators': 500}
[7/180] {'eta': 0.3, 'eval_metric': 'rmse', 'gamma': 0.3, 'max_depth': 3, 'min_child_weight': 5, 'n_estimators': 250}
[8/180] {'eta': 0.3, 'eval_metric': 'rmse', 'gamma': 0.3, 'max_depth': 3, 'min_child_weight': 5, 'n_estimators': 500}
[9/180] {'eta': 0.3, 'eval_metric': 'rmse', 'gamma': 0.3

### LGBM

In [184]:
Model_Name = "LGBM"

# Define Grid Search parameters
parameters = {'learning_rate':[0.01, 0.05, 0.1, 0.5, 0.9],'verbosity':[-1]}

grid = ParameterGrid(parameters)

lower_rmse = np.inf
for i,param in enumerate(grid):
    
    print(f"[{i+1}/{len(grid)}] {param}" )
    
    # Optimize parameters
    LGBM = LGBMRegressor(**param)
    LGBM.fit(X_train,y_train.ravel())
    # Make predictions
    y_pred = LGBM.predict(X_val)
    
    
    # Calculating the error metrics
    # Compute the Root Mean Square Error
    RMSE = math.sqrt(mean_squared_error(y_val, y_pred))
    
    if RMSE < lower_rmse:
        lower_rmse = RMSE
        best_LGBM_params = param

# Optimized parameters

new_X_train, new_y_train = X.values[:index_val], y[:index_val]
LGBM = LGBMRegressor(**best_LGBM_params)
LGBM.fit(new_X_train,new_y_train)
# Make predictions
y_pred = LGBM.predict(X_test)

# Save predictions to dataframe
y_pred = scaler_y.inverse_transform(y_pred.reshape(1, -1)).reshape(-1,1)
predictions[f'{Model_Name}'] = y_pred

# Model name
print(f'\nResults for {Model_Name}: \n')

# Calculating the error metrics
# Compute the Root Mean Square Error
RMSE = math.sqrt(mean_squared_error(y_test, y_pred))
print("\nRMSE:", RMSE)
# Compute the Normalized Root Mean Square Error
NRMSE = RegressionMetric(y_test, y_pred).normalized_root_mean_square_error()
print("NRMSE:", NRMSE)
# Compute the Non-Dimensional Error Index
NDEI= RMSE/st.stdev(np.asfarray(y_test.flatten()))
print("NDEI:", NDEI)
# Compute the Mean Absolute Error
MAE = mean_absolute_error(y_test, y_pred)
print("MAE:", MAE)
# Compute the Mean Absolute Percentage Error
MAPE = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE:", MAPE)
# Number of rules
Rules = "-"
print("Rules:", Rules)
   
LGBM_ = f'{Model_Name} & {NRMSE:.2f} & {NDEI:.2f} & {MAPE:.2f} & {Rules}'

print(f'\n{best_LGBM_params}')

# Store results to dataframe
newrow = pd.DataFrame([[Model_Name, NRMSE, NDEI, MAPE, Rules, best_LGBM_params]], columns=columns)
results = pd.concat([results, newrow], ignore_index=True)

# salvando em planilha
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")

results.to_excel(f"../Relatórios/results_{timestamp}.xlsx")
predictions.to_excel(f"../Relatórios/predictions_{timestamp}.xlsx")

[1/5] {'learning_rate': 0.01, 'verbosity': -1}
[2/5] {'learning_rate': 0.05, 'verbosity': -1}
[3/5] {'learning_rate': 0.1, 'verbosity': -1}
[4/5] {'learning_rate': 0.5, 'verbosity': -1}
[5/5] {'learning_rate': 0.9, 'verbosity': -1}


C:\Users\thevh\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Results for LGBM: 


RMSE: 0.5974629570775071
NRMSE: 0.22886810975381366
NDEI: 0.21201043175175668
MAE: 0.3819869035403932
MAPE: 0.31449419321918587
Rules: -

{'learning_rate': 0.05, 'verbosity': -1}


## Deep Learning

In [190]:
X_train_DL = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_val_DL = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)
X_test_DL = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [202]:
Model_Name = "MLP"

# Define the function to create models for the optimization method
def build_model(n_hidden=1, n_neurons=30, activation = "relu", learning_rate=3e-3, input_shape=[4]):
    model = Sequential()
    model.add(InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(Dense(n_neurons, activation=activation))
    model.add(Dense(1))
    optimizer = SGD(learning_rate=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

# Set the parameters for the network
best_MLP_parameters = {'n_hidden':2, 'n_neurons':77, 'activation':"relu", 'learning_rate': 0.145, 'input_shape': X_train_DL.shape[1]}

# Checkpoint for early stopping
checkpoint_cb = ModelCheckpoint(f'RandomSearchResults/{Model_Name}.keras', save_best_only=True)
early_stopping_cb = EarlyStopping(patience=20,restore_best_weights=True)

# Define the model
model = build_model(**best_MLP_parameters)

# Fit the model
history = model.fit(X_train_DL, y_train, epochs = 1000, validation_data=(X_val_DL, y_val), callbacks=[checkpoint_cb, early_stopping_cb])

# Make predictions
y_pred = model.predict(X_test_DL)



c:\Users\thevh\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


ValueError: Cannot convert '12' to a shape.

In [ ]:
# Save predictions to dataframe
y_pred = scaler_y.inverse_transform(y_pred.reshape(1, -1)).reshape(-1,1)
predictions[f'{Model_Name}'] = y_pred.ravel()

# Model name
print(f'\nResults for {Model_Name}: \n')

# Calculating the error metrics
# Compute the Root Mean Square Error
RMSE = math.sqrt(mean_squared_error(y_test, y_pred))
print("\nRMSE:", RMSE)
# Compute the Normalized Root Mean Square Error
NRMSE = RegressionMetric(y_test, y_pred).normalized_root_mean_square_error()
print("NRMSE:", NRMSE)
# Compute the Non-Dimensional Error Index
NDEI= RMSE/st.stdev(np.asfarray(y_test.flatten()))
print("NDEI:", NDEI)
# Compute the Mean Absolute Error
MAE = mean_absolute_error(y_test, y_pred)
print("MAE:", MAE)
# Compute the Mean Absolute Percentage Error
MAPE = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE:", MAPE)
# Number of rules
Rules = "-"
print("Rules:", Rules)
   
MLP_ = f'{Model_Name} & {NRMSE:.2f} & {NDEI:.2f} & {MAPE:.2f} & {Rules}'

# Store results to dataframe
newrow = pd.DataFrame([[Model_Name, NRMSE, NDEI, MAPE, Rules, best_MLP_parameters]], columns=columns)
results = pd.concat([results, newrow], ignore_index=True)


## evolving Fuzzy Systems

### eTS

In [205]:
Model_Name = "eTS"

# Define Grid Search parameters
# parameters = {'InitialOmega': [50, 100, 250, 500, 750, 1000, 10000], 'r': [0.1, 0.3, 0.5, 0.7, 0.9, 1., 5, 10, 50]}

parameters = {'InitialOmega': [50], 'r': [0.1]}

grid = ParameterGrid(parameters)

lower_rmse = np.inf
for i,param in enumerate(grid):
    
    print(f"[{i+1}/{len(grid)}] {param}" )

    # Optimize parameters
    model = eTS(**param)
    model.fit(X_train,y_train)
    # Make predictions
    y_pred = model.predict(X_val)
    
    
    # Calculating the error metrics
    # Compute the Root Mean Square Error
    RMSE = math.sqrt(mean_squared_error(y_val, y_pred))
    
    if RMSE < lower_rmse:
        lower_rmse = RMSE
        best_eTS_params = param

# Optimized parameters

new_X_train, new_y_train = X.values[:index_val], y[:index_val]
model = eTS(**best_eTS_params)
OutputTraining, Rules = model.fit(new_X_train,new_y_train)
# Make predictions
y_pred = model.predict(X_test)

# Save predictions to dataframe
y_pred = scaler_y.inverse_transform(y_pred.reshape(1, -1)).reshape(-1,1)
predictions[f'{Model_Name}'] = y_pred

# Model name
print(f'\nResults for {Model_Name}: \n')

# Calculating the error metrics
# Compute the Root Mean Square Error
RMSE = math.sqrt(mean_squared_error(y_test, y_pred))
print("\nRMSE:", RMSE)
# Compute the Normalized Root Mean Square Error
NRMSE = RegressionMetric(y_test, y_pred).normalized_root_mean_square_error()
print("NRMSE:", NRMSE)
# Compute the Non-Dimensional Error Index
NDEI= RMSE/st.stdev(np.asfarray(y_test.flatten()))
print("NDEI:", NDEI)
# Compute the Mean Absolute Error
MAE = mean_absolute_error(y_test, y_pred)
print("MAE:", MAE)
# Compute the Mean Absolute Percentage Error
MAPE = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE:", MAPE)
# Number of rules
print("Rules:", Rules[-1])

   
eTS_ = f'{Model_Name} & {NRMSE:.2f} & {NDEI:.2f} & {MAPE:.2f} & {Rules[-1]}'

print(f'\n{best_eTS_params}')

# Store results to dataframe
newrow = pd.DataFrame([[Model_Name, NRMSE, NDEI, MAPE, Rules[-1], best_eTS_params]], columns=columns)
results = pd.concat([results, newrow], ignore_index=True)

# salvando em planilha
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")

results.to_excel(f"../Relatórios/results_{timestamp}.xlsx")
predictions.to_excel(f"../Relatórios/predictions_{timestamp}.xlsx")

[1/1] {'InitialOmega': 50, 'r': 0.1}

Results for eTS: 


RMSE: 0.6453815145937141
NRMSE: 0.2336932825308349
NDEI: 0.2290143881436749
MAE: 0.3601403476494569
MAPE: 0.1911948655404788
Rules: 4

{'InitialOmega': 50, 'r': 0.1}


### Simpl_eTS

In [208]:
Model_Name = "Simpl_eTS"

# Define Grid Search parameters
# parameters = {'InitialOmega': [50, 250, 500, 750, 1000], 'r': [0.1, 0.3, 0.5, 0.7]}
parameters = {'InitialOmega': [50], 'r': [0.1]}

grid = ParameterGrid(parameters)

lower_rmse = np.inf
for i,param in enumerate(grid):
    
    print(f"[{i+1}/{len(grid)}] {param}" )

    # Optimize parameters
    model = Simpl_eTS(**param)
    model.fit(X_train,y_train)
    # Make predictions
    y_pred = model.predict(X_val)
    
    
    # Calculating the error metrics
    # Compute the Root Mean Square Error
    RMSE = math.sqrt(mean_squared_error(y_val, y_pred))
    
    if RMSE < lower_rmse:
        lower_rmse = RMSE
        best_Simpl_eTS_params = param

# Optimized parameters

new_X_train, new_y_train = X.values[:index_val], y[:index_val]
model = Simpl_eTS(**best_Simpl_eTS_params)
OutputTraining, Rules = model.fit(new_X_train,new_y_train)
# Make predictions
y_pred = model.predict(X_test)

# Save predictions to dataframe
y_pred = scaler_y.inverse_transform(y_pred.reshape(1, -1)).reshape(-1,1)
predictions[f'{Model_Name}'] = y_pred

# Model name
print(f'\nResults for {Model_Name}: \n')

# Calculating the error metrics
# Compute the Root Mean Square Error
RMSE = math.sqrt(mean_squared_error(y_test, y_pred))
print("\nRMSE:", RMSE)
# Compute the Normalized Root Mean Square Error
NRMSE = RegressionMetric(y_test, y_pred).normalized_root_mean_square_error()
print("NRMSE:", NRMSE)
# Compute the Non-Dimensional Error Index
NDEI= RMSE/st.stdev(np.asfarray(y_test.flatten()))
print("NDEI:", NDEI)
# Compute the Mean Absolute Error
MAE = mean_absolute_error(y_test, y_pred)
print("MAE:", MAE)
# Compute the Mean Absolute Percentage Error
MAPE = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE:", MAPE)
# Number of rules
print("Rules:", Rules[-1])
   
Simpl_eTS_ = f'{Model_Name} & {NRMSE:.2f} & {NDEI:.2f} & {MAPE:.2f} & {Rules[-1]}'

print(f'\n{best_Simpl_eTS_params}')

# Store results to dataframe
newrow = pd.DataFrame([[Model_Name, NRMSE, NDEI, MAPE, Rules[-1], best_Simpl_eTS_params]], columns=columns)
results = pd.concat([results, newrow], ignore_index=True)

# salvando em planilha
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")

results.to_excel(f"../Relatórios/results_{timestamp}.xlsx")
predictions.to_excel(f"../Relatórios/predictions_{timestamp}.xlsx")

[1/1] {'InitialOmega': 50, 'r': 0.1}

Results for Simpl_eTS: 


RMSE: 0.8876415158524629
NRMSE: 0.3224989795764989
NDEI: 0.31498063400816195
MAE: 0.5204420813679302
MAPE: 0.2665403667925537
Rules: 34

{'InitialOmega': 50, 'r': 0.1}


### exTS

In [ ]:
Model_Name = "exTS"

# Define Grid Search parameters
parameters = {'InitialOmega': [50, 250, 500, 750, 1000], 'mu_threshold': [0.1, 0.3, 0.5, 0.7]}
grid = ParameterGrid(parameters)

lower_rmse = np.inf
for i,param in enumerate(grid):
    
    print(f"[{i+1}/{len(grid)}] {param}" )

    # Optimize parameters
    model = exTS(**param)
    model.fit(X_train,y_train)
    # Make predictions
    y_pred = model.predict(X_val)
    
    
    # Calculating the error metrics
    # Compute the Root Mean Square Error
    RMSE = math.sqrt(mean_squared_error(y_val, y_pred))
    
    if RMSE < lower_rmse:
        lower_rmse = RMSE
        best_exTS_params = param

# Optimized parameters

new_X_train, new_y_train = X.values[:index_val], y[:index_val]
model = exTS(**best_exTS_params)
OutputTraining, Rules = model.fit(new_X_train,new_y_train)
# Make predictions
y_pred = model.predict(X_test)

# Save predictions to dataframe
y_pred = scaler_y.inverse_transform(y_pred.reshape(1, -1)).reshape(-1,1)
predictions[f'{Model_Name}'] = y_pred

# Model name
print(f'\nResults for {Model_Name}: \n')

# Calculating the error metrics
# Compute the Root Mean Square Error
RMSE = math.sqrt(mean_squared_error(y_test, y_pred))
print("\nRMSE:", RMSE)
# Compute the Normalized Root Mean Square Error
NRMSE = RegressionMetric(y_test, y_pred).normalized_root_mean_square_error()
print("NRMSE:", NRMSE)
# Compute the Non-Dimensional Error Index
NDEI= RMSE/st.stdev(np.asfarray(y_test.flatten()))
print("NDEI:", NDEI)
# Compute the Mean Absolute Error
MAE = mean_absolute_error(y_test, y_pred)
print("MAE:", MAE)
# Compute the Mean Absolute Percentage Error
MAPE = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE:", MAPE)
# Number of rules
print("Rules:", Rules[-1])
   
exTS_ = f'{Model_Name} & {NRMSE:.2f} & {NDEI:.2f} & {MAPE:.2f} & {Rules[-1]}'

print(f'\n{best_exTS_params}')

# Store results to dataframe
newrow = pd.DataFrame([[Model_Name, NRMSE, NDEI, MAPE, Rules[-1], best_exTS_params]], columns=columns)
results = pd.concat([results, newrow], ignore_index=True)

# salvando em planilha
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")

results.to_excel(f"../Relatórios/results_{timestamp}.xlsx")
predictions.to_excel(f"../Relatórios/predictions_{timestamp}.xlsx")


### ePL

In [ ]:
Model_Name = "ePL"

# Define Grid Search parameters
parameters = {'alpha': [0.001, 0.01, 0.1, 0.5, 0.9], 'beta': [0.001, 0.005, 0.01, 0.1, 0.2], 'lambda1': [0.001, 0.01, 0.1], 's': [100, 10000], 'r': [0.1, 0.25, 0.5, 0.75]}
grid = ParameterGrid(parameters)

lower_rmse = np.inf
for i,param in enumerate(grid):
    
    print(f"[{i+1}/{len(grid)}] {param}" )

    # Optimize parameters
    model = ePL(**param)
    model.fit(X_train,y_train)
    # Make predictions
    y_pred = model.predict(X_val)
    
    
    # Calculating the error metrics
    # Compute the Root Mean Square Error
    RMSE = math.sqrt(mean_squared_error(y_val, y_pred))
    
    if RMSE < lower_rmse:
        lower_rmse = RMSE
        best_ePL_params = param

# Optimized parameters

new_X_train, new_y_train = X.values[:index_val], y[:index_val]
model = ePL(**best_ePL_params)
OutputTraining, Rules = model.fit(new_X_train,new_y_train)
# Make predictions
y_pred = model.predict(X_test)

# Save predictions to dataframe
y_pred = scaler_y.inverse_transform(y_pred.reshape(1, -1)).reshape(-1,1)
predictions[f'{Model_Name}'] = y_pred

# Model name
print(f'\nResults for {Model_Name}: \n')

# Calculating the error metrics
# Compute the Root Mean Square Error
RMSE = math.sqrt(mean_squared_error(y_test, y_pred))
print("\nRMSE:", RMSE)
# Compute the Normalized Root Mean Square Error
NRMSE = RegressionMetric(y_test, y_pred).normalized_root_mean_square_error()
print("NRMSE:", NRMSE)
# Compute the Non-Dimensional Error Index
NDEI= RMSE/st.stdev(np.asfarray(y_test.flatten()))
print("NDEI:", NDEI)
# Compute the Mean Absolute Error
MAE = mean_absolute_error(y_test, y_pred)
print("MAE:", MAE)
# Compute the Mean Absolute Percentage Error
MAPE = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE:", MAPE)
# Number of rules
print("Rules:", Rules[-1])
   
ePL_ = f'{Model_Name} & {NRMSE:.2f} & {NDEI:.2f} & {MAPE:.2f} & {Rules[-1]}'

print(f'\n{best_ePL_params}')

# Store results to dataframe
newrow = pd.DataFrame([[Model_Name, NRMSE, NDEI, MAPE, Rules[-1], best_ePL_params]], columns=columns)
results = pd.concat([results, newrow], ignore_index=True)

# salvando em planilha
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")

results.to_excel(f"../Relatórios/results_{timestamp}.xlsx")
predictions.to_excel(f"../Relatórios/predictions_{timestamp}.xlsx")

### eMG

In [ ]:
Model_Name = "eMG"

# Define Grid Search parameters
parameters = {'alpha': [0.001, 0.01], 'lambda1': [0.1, 0.5], 'w': [10, 50], 'sigma': [0.001, 0.003], 'omega': [10**4]}
grid = ParameterGrid(parameters)

lower_rmse = np.inf
for i,param in enumerate(grid):
    
    print(f"[{i+1}/{len(grid)}] {param}" )

    # Optimize parameters
    model = eMG(**param)
    model.fit(X_train,y_train)
    # Make predictions
    y_pred = model.predict(X_val)
    
    
    # Calculating the error metrics
    # Compute the Root Mean Square Error
    RMSE = math.sqrt(mean_squared_error(y_val, y_pred))
    
    if RMSE < lower_rmse:
        lower_rmse = RMSE
        best_eMG_params = param

# Optimized parameters

new_X_train, new_y_train = X.values[:index_val], y[:index_val]
model = eMG(**best_eMG_params)
OutputTraining, Rules = model.fit(new_X_train,new_y_train)
# Make predictions
y_pred = model.predict(X_test)

# Save predictions to dataframe
y_pred = scaler_y.inverse_transform(y_pred.reshape(1, -1)).reshape(-1,1)
predictions[f'{Model_Name}'] = y_pred

# Model name
print(f'\nResults for {Model_Name}: \n')

# Calculating the error metrics
# Compute the Root Mean Square Error
RMSE = math.sqrt(mean_squared_error(y_test, y_pred))
print("\nRMSE:", RMSE)
# Compute the Normalized Root Mean Square Error
NRMSE = RegressionMetric(y_test, y_pred).normalized_root_mean_square_error()
print("NRMSE:", NRMSE)
# Compute the Non-Dimensional Error Index
NDEI= RMSE/st.stdev(np.asfarray(y_test.flatten()))
print("NDEI:", NDEI)
# Compute the Mean Absolute Error
MAE = mean_absolute_error(y_test, y_pred)
print("MAE:", MAE)
# Compute the Mean Absolute Percentage Error
MAPE = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE:", MAPE)
# Number of rules
print("Rules:", Rules[-1])
   
eMG_ = f'{Model_Name} & {NRMSE:.2f} & {NDEI:.2f} & {MAPE:.2f} & {Rules[-1]}'

print(f'\n{best_eMG_params}')

# Store results to dataframe
newrow = pd.DataFrame([[Model_Name, NRMSE, NDEI, MAPE, Rules[-1], best_eMG_params]], columns=columns)
results = pd.concat([results, newrow], ignore_index=True)

# salvando em planilha
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")

results.to_excel(f"../Relatórios/results_{timestamp}.xlsx")
predictions.to_excel(f"../Relatórios/predictions_{timestamp}.xlsx")


### ePL+

In [ ]:
Model_Name = "ePL_plus"

# Define Grid Search parameters
parameters = {'alpha': [0.001, 0.01, 0.1], 'beta': [0.01, 0.1, 0.25], 'lambda1': [0.25, 0.5, 0.75], 'omega': [100, 10000], 'sigma': [0.1, 0.25, 0.5], 'e_Utility': [0.03, 0.05], 'pi': [0.3, 0.5]}
grid = ParameterGrid(parameters)

lower_rmse = np.inf
for i,param in enumerate(grid):
    
    print(f"[{i+1}/{len(grid)}] {param}" )

    # Optimize parameters
    model = ePL_plus(**param)
    model.fit(X_train,y_train)
    # Make predictions
    y_pred = model.predict(X_val)
    
    
    # Calculating the error metrics
    # Compute the Root Mean Square Error
    RMSE = math.sqrt(mean_squared_error(y_val, y_pred))
    
    if RMSE < lower_rmse:
        lower_rmse = RMSE
        best_ePL_plus_params = param

# Optimized parameters

new_X_train, new_y_train = X.values[:index_val], y[:index_val]
model = ePL_plus(**best_ePL_plus_params)
OutputTraining, Rules = model.fit(new_X_train,new_y_train)
# Make predictions
y_pred = model.predict(X_test)

# Save predictions to dataframe
y_pred = scaler_y.inverse_transform(y_pred.reshape(1, -1)).reshape(-1,1)
predictions[f'{Model_Name}'] = y_pred

# Model name
print(f'\nResults for {Model_Name}: \n')

# Calculating the error metrics
# Compute the Root Mean Square Error
RMSE = math.sqrt(mean_squared_error(y_test, y_pred))
print("\nRMSE:", RMSE)
# Compute the Normalized Root Mean Square Error
NRMSE = RegressionMetric(y_test, y_pred).normalized_root_mean_square_error()
print("NRMSE:", NRMSE)
# Compute the Non-Dimensional Error Index
NDEI= RMSE/st.stdev(np.asfarray(y_test.flatten()))
print("NDEI:", NDEI)
# Compute the Mean Absolute Error
MAE = mean_absolute_error(y_test, y_pred)
print("MAE:", MAE)
# Compute the Mean Absolute Percentage Error
MAPE = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE:", MAPE)
# Number of rules
print("Rules:", Rules[-1])
   
ePL_plus_ = f'{Model_Name} & {NRMSE:.2f} & {NDEI:.2f} & {MAPE:.2f} & {Rules[-1]}'

print(f'\n{best_ePL_plus_params}')

# Store results to dataframe
newrow = pd.DataFrame([[Model_Name, NRMSE, NDEI, MAPE, Rules[-1], best_ePL_plus_params]], columns=columns)
results = pd.concat([results, newrow], ignore_index=True)

# salvando em planilha
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")

results.to_excel(f"../Relatórios/results_{timestamp}.xlsx")
predictions.to_excel(f"../Relatórios/predictions_{timestamp}.xlsx")


### ePL-KRLS-DISCO

In [ ]:
Model_Name = "ePL_KRLS_DISCO"

# Define Grid Search parameters
parameters = {'alpha': [0.05, 0.1], 'beta': [0.01, 0.1, 0.25], 'lambda1': [0.0000001, 0.001], 'sigma': [0.5, 1, 10, 50], 'e_utility': [0.03, 0.05]}
grid = ParameterGrid(parameters)

lower_rmse = np.inf
for i,param in enumerate(grid):
    
    print(f"[{i+1}/{len(grid)}] {param}" )

    # Optimize parameters
    model = ePL_KRLS_DISCO(**param)
    model.fit(X_train,y_train)
    # Make predictions
    y_pred = model.predict(X_val)
    
    
    # Calculating the error metrics
    # Compute the Root Mean Square Error
    RMSE = math.sqrt(mean_squared_error(y_val, y_pred))
    
    if RMSE < lower_rmse:
        lower_rmse = RMSE
        best_ePL_KRLS_DISCO_params = param
        
# Optimized parameters

new_X_train, new_y_train = X.values[:index_val], y[:index_val]
model = ePL_KRLS_DISCO(**best_ePL_KRLS_DISCO_params)
OutputTraining, Rules = model.fit(new_X_train,new_y_train)
# Make predictions
y_pred = model.predict(X_test)

# Save predictions to dataframe
y_pred = scaler_y.inverse_transform(y_pred.reshape(1, -1)).reshape(-1,1)
predictions[f'{Model_Name}'] = y_pred

# Model name
print(f'\nResults for {Model_Name}: \n')

# Calculating the error metrics
# Compute the Root Mean Square Error
RMSE = math.sqrt(mean_squared_error(y_test, y_pred))
print("\nRMSE:", RMSE)
# Compute the Normalized Root Mean Square Error
NRMSE = RegressionMetric(y_test, y_pred).normalized_root_mean_square_error()
print("NRMSE:", NRMSE)
# Compute the Non-Dimensional Error Index
NDEI= RMSE/st.stdev(np.asfarray(y_test.flatten()))
print("NDEI:", NDEI)
# Compute the Mean Absolute Error
MAE = mean_absolute_error(y_test, y_pred)
print("MAE:", MAE)
# Compute the Mean Absolute Percentage Error
MAPE = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE:", MAPE)
# Number of rules
print("Rules:", Rules[-1])
   
ePL_KRLS_DISCO_ = f'{Model_Name} & {NRMSE:.2f} & {NDEI:.2f} & {MAPE:.2f} & {Rules[-1]}'

#print(f'\n{best_ePL_KRLS_DISCO_params}')

# Store results to dataframe
newrow = pd.DataFrame([[Model_Name, NRMSE, NDEI, MAPE, Rules[-1], best_ePL_KRLS_DISCO_params]], columns=columns)
results = pd.concat([results, newrow], ignore_index=True)

# salvando em planilha
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")

results.to_excel(f"../Relatórios/results_{timestamp}.xlsx")
predictions.to_excel(f"../Relatórios/predictions_{timestamp}.xlsx")

## Proposed Models (Kaike)

### NMFIS

In [ ]:
Model_Name = "NMFIS"

# Set hyperparameters range
parameters = {'n_clusters':range(1,20)}

grid = ParameterGrid(parameters)

lower_rmse = np.inf
for i,param in enumerate(grid):
    
    print(f"[{i+1}/{len(grid)}] {param}" )

    # Optimize parameters
    model = NMFIS(**param)
    model.fit(X_train,y_train)
    # Make predictions
    y_pred = model.predict(X_val)
    
    
    # Calculating the error metrics
    # Compute the Root Mean Square Error
    RMSE = math.sqrt(mean_squared_error(y_val, y_pred))
    
    if RMSE < lower_rmse:
        lower_rmse = RMSE
        best_NMFIS_params = param

# Initialize the model
model = NMFIS(**best_NMFIS_params)
new_X_train, new_y_train = X.values[:index_val], y[:index_val]

# Train the model
OutputTraining = model.fit(new_X_train, new_y_train)
# Test the model
y_pred = model.predict(X_test)

# Save predictions to dataframe
y_pred = scaler_y.inverse_transform(y_pred.reshape(1, -1)).reshape(-1,1)
predictions[f'{Model_Name}'] = y_pred

# Model name
print(f'\nResults for {Model_Name}: \n')

# Calculating the error metrics
# Compute the Root Mean Square Error
RMSE = math.sqrt(mean_squared_error(y_test, y_pred))
print("\nRMSE:", RMSE)
# Compute the Normalized Root Mean Square Error
NRMSE = RegressionMetric(y_test, y_pred).normalized_root_mean_square_error()
print("NRMSE:", NRMSE)
# Compute the Non-Dimensional Error Index
NDEI= RMSE/st.stdev(np.asfarray(y_test.flatten()))
print("NDEI:", NDEI)
# Compute the Mean Absolute Error
MAE = mean_absolute_error(y_test, y_pred)
print("MAE:", MAE)
# Compute the Mean Absolute Percentage Error
MAPE = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE:", MAPE)
# Compute the number of final rules
Rules = model.parameters.shape[0]
print("Rules:", Rules)



NMFIS_ = f'{Model_Name} & {NRMSE:.2f} & {NDEI:.2f} & {MAPE:.2f} & {Rules}'

# Store results to dataframe
newrow = pd.DataFrame([[Model_Name, NRMSE, NDEI, MAPE, Rules, best_NMFIS_params]], columns=columns)
results = pd.concat([results, newrow], ignore_index=True)

# salvando em planilha
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")

results.to_excel(f"../Relatórios/results_{timestamp}.xlsx")
predictions.to_excel(f"../Relatórios/predictions_{timestamp}.xlsx")

try:
    # Save  model to excel
    model.parameters.to_excel(f'Model Summary/{Model_Name}_parameters.xlsx')

### NTSK-RLS

In [219]:
Model_Name = "NTSK-RLS"

# Set hyperparameters range
parameters = {'n_clusters':[1], 'lambda1':[0.95,0.96,0.97,0.98,0.99], 'RLS_option':[1]}

grid = ParameterGrid(parameters)

lower_rmse = np.inf
for i,param in enumerate(grid):
    
    print(f"[{i+1}/{len(grid)}] {param}" )

    # Optimize parameters
    model = NTSK(**param)
    model.fit(X_train,y_train)
    # Make predictions
    y_pred = model.predict(X_val)
    
    
    # Calculating the error metrics
    # Compute the Root Mean Square Error
    RMSE = math.sqrt(mean_squared_error(y_val, y_pred))
    
    if RMSE < lower_rmse:
        lower_rmse = RMSE
        best_NTSK_RLS_params = param

# Initialize the model
model = NTSK(**best_NTSK_RLS_params)
new_X_train, new_y_train = X.values[:index_val], y[:index_val]

# Train the model
OutputTraining = model.fit(new_X_train, new_y_train)
# Test the model
y_pred = model.predict(X_test)

# Save predictions to dataframe
y_pred = scaler_y.inverse_transform(y_pred.reshape(1, -1)).reshape(-1,1)
predictions[f'{Model_Name}'] = y_pred

# Model name
print(f'\nResults for {Model_Name}: \n')

# Calculating the error metrics
# Compute the Root Mean Square Error
RMSE = math.sqrt(mean_squared_error(y_test, y_pred))
print("\nRMSE:", RMSE)
# Compute the Normalized Root Mean Square Error
NRMSE = RegressionMetric(y_test, y_pred).normalized_root_mean_square_error()
print("NRMSE:", NRMSE)
# Compute the Non-Dimensional Error Index
NDEI= RMSE/st.stdev(np.asfarray(y_test.flatten()))
print("NDEI:", NDEI)
# Compute the Mean Absolute Error
MAE = mean_absolute_error(y_test, y_pred)
print("MAE:", MAE)
# Compute the Mean Absolute Percentage Error
MAPE = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE:", MAPE)
# Compute the number of final rules
Rules = model.parameters.shape[0]
print("Rules:", Rules)  

NTSK_RLS_ = f'{Model_Name} & {NRMSE:.2f} & {NDEI:.2f} & {MAPE:.2f} & {Rules}'



# Store results to dataframe
newrow = pd.DataFrame([[Model_Name, NRMSE, NDEI, MAPE, Rules, best_NTSK_RLS_params]], columns=columns)
results = pd.concat([results, newrow], ignore_index=True)

# salvando em planilha
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")

results.to_excel(f"../Relatórios/results_{timestamp}.xlsx")
predictions.to_excel(f"../Relatórios/predictions_{timestamp}.xlsx")

try:
    # Save the model to excel
    model.parameters.to_excel(f'../Relatórios/Model Summary/{Model_Name}_parameters.xlsx')
except Exception as e:
    print(f"An error occurred while saving the model parameters: {e}")


[1/5] {'RLS_option': 1, 'lambda1': 0.95, 'n_clusters': 1}


d:\Dissertação\dissertacao_pgmc\Cadernos\..\Modelo\NTSK.py:102: RuntimeWarning: overflow encountered in square
  self.ResidualTrainingPhase = np.append(self.ResidualTrainingPhase,(Output - y[k])**2)
d:\Dissertação\dissertacao_pgmc\Cadernos\..\Modelo\NTSK.py:188: RuntimeWarning: overflow encountered in multiply
  self.Theta = self.Theta + ( self.P @ xe ) * (y - xe.T @ self.Theta )
d:\Dissertação\dissertacao_pgmc\Cadernos\..\Modelo\NTSK.py:188: RuntimeWarning: invalid value encountered in multiply
  self.Theta = self.Theta + ( self.P @ xe ) * (y - xe.T @ self.Theta )
d:\Dissertação\dissertacao_pgmc\Cadernos\..\Modelo\NTSK.py:188: RuntimeWarning: invalid value encountered in add
  self.Theta = self.Theta + ( self.P @ xe ) * (y - xe.T @ self.Theta )
d:\Dissertação\dissertacao_pgmc\Cadernos\..\Modelo\NTSK.py:95: RuntimeWarning: invalid value encountered in matmul
  Output =  xe.T @ self.Theta
d:\Dissertação\dissertacao_pgmc\Cadernos\..\Modelo\NTSK.py:188: RuntimeWarning: invalid value encou

[2/5] {'RLS_option': 1, 'lambda1': 0.96, 'n_clusters': 1}


d:\Dissertação\dissertacao_pgmc\Cadernos\..\Modelo\NTSK.py:102: RuntimeWarning: overflow encountered in square
  self.ResidualTrainingPhase = np.append(self.ResidualTrainingPhase,(Output - y[k])**2)
d:\Dissertação\dissertacao_pgmc\Cadernos\..\Modelo\NTSK.py:188: RuntimeWarning: overflow encountered in multiply
  self.Theta = self.Theta + ( self.P @ xe ) * (y - xe.T @ self.Theta )
d:\Dissertação\dissertacao_pgmc\Cadernos\..\Modelo\NTSK.py:95: RuntimeWarning: overflow encountered in matmul
  Output =  xe.T @ self.Theta
d:\Dissertação\dissertacao_pgmc\Cadernos\..\Modelo\NTSK.py:95: RuntimeWarning: invalid value encountered in matmul
  Output =  xe.T @ self.Theta
d:\Dissertação\dissertacao_pgmc\Cadernos\..\Modelo\NTSK.py:188: RuntimeWarning: overflow encountered in matmul
  self.Theta = self.Theta + ( self.P @ xe ) * (y - xe.T @ self.Theta )
d:\Dissertação\dissertacao_pgmc\Cadernos\..\Modelo\NTSK.py:188: RuntimeWarning: invalid value encountered in matmul
  self.Theta = self.Theta + ( self

[3/5] {'RLS_option': 1, 'lambda1': 0.97, 'n_clusters': 1}


d:\Dissertação\dissertacao_pgmc\Cadernos\..\Modelo\NTSK.py:102: RuntimeWarning: overflow encountered in square
  self.ResidualTrainingPhase = np.append(self.ResidualTrainingPhase,(Output - y[k])**2)
d:\Dissertação\dissertacao_pgmc\Cadernos\..\Modelo\NTSK.py:188: RuntimeWarning: overflow encountered in multiply
  self.Theta = self.Theta + ( self.P @ xe ) * (y - xe.T @ self.Theta )
d:\Dissertação\dissertacao_pgmc\Cadernos\..\Modelo\NTSK.py:95: RuntimeWarning: overflow encountered in matmul
  Output =  xe.T @ self.Theta
d:\Dissertação\dissertacao_pgmc\Cadernos\..\Modelo\NTSK.py:95: RuntimeWarning: invalid value encountered in matmul
  Output =  xe.T @ self.Theta
d:\Dissertação\dissertacao_pgmc\Cadernos\..\Modelo\NTSK.py:188: RuntimeWarning: overflow encountered in matmul
  self.Theta = self.Theta + ( self.P @ xe ) * (y - xe.T @ self.Theta )
d:\Dissertação\dissertacao_pgmc\Cadernos\..\Modelo\NTSK.py:188: RuntimeWarning: invalid value encountered in matmul
  self.Theta = self.Theta + ( self

[4/5] {'RLS_option': 1, 'lambda1': 0.98, 'n_clusters': 1}
[5/5] {'RLS_option': 1, 'lambda1': 0.99, 'n_clusters': 1}


KeyboardInterrupt: 

### NTSK-wRLS

In [225]:
Model_Name = "NTSK-wRLS"

# Set hyperparameters range
# parameters = {'n_clusters':range(1,20), 'RLS_option':[2]}
parameters = {'n_clusters':range(1,2), 'RLS_option':[2]}

grid = ParameterGrid(parameters)

lower_rmse = np.inf
for i,param in enumerate(grid):
    
    print(f"[{i+1}/{len(grid)}] {param}")

    # Optimize parameters
    model = NTSK(**param)
    model.fit(X_train,y_train)
    # Make predictions
    y_pred = model.predict(X_val)
    
    
    # Calculating the error metrics
    # Compute the Root Mean Square Error
    RMSE = math.sqrt(mean_squared_error(y_val, y_pred))
    
    if RMSE < lower_rmse:
        lower_rmse = RMSE
        best_NTSK_wRLS_params = param

# Initialize the model
model = NTSK(**best_NTSK_wRLS_params)
# Train the model
new_X_train, new_y_train = X.values[:index_val], y[:index_val]
OutputTraining = model.fit(new_X_train, new_y_train)
# Test the model
y_pred = model.predict(X_test)

# Save predictions to dataframe
y_pred = scaler_y.inverse_transform(y_pred.reshape(1, -1)).reshape(-1,1)
predictions[f'{Model_Name}'] = y_pred

# Model name
print(f'\nResults for {Model_Name}: \n')

# Calculating the error metrics
# Compute the Root Mean Square Error
RMSE = math.sqrt(mean_squared_error(y_test, y_pred))
print("\nRMSE:", RMSE)
# Compute the Normalized Root Mean Square Error
NRMSE = RegressionMetric(y_test, y_pred).normalized_root_mean_square_error()
print("NRMSE:", NRMSE)
# Compute the Non-Dimensional Error Index
NDEI= RMSE/st.stdev(np.asfarray(y_test.flatten()))
print("NDEI:", NDEI)
# Compute the Mean Absolute Error
MAE = mean_absolute_error(y_test, y_pred)
print("MAE:", MAE)
# Compute the Mean Absolute Percentage Error
MAPE = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE:", MAPE)
# Compute the number of final rules
Rules = model.parameters.shape[0]
print("Rules:", Rules)  



NTSK_wRLS_ = f'{Model_Name} & {NRMSE:.2f} & {NDEI:.2f} & {MAPE:.2f} & {Rules}'

# Store results to dataframe
newrow = pd.DataFrame([[Model_Name, NRMSE, NDEI, MAPE, Rules, best_NTSK_wRLS_params]], columns=columns)
results = pd.concat([results, newrow], ignore_index=True)

# salvando em planilha
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")

results.to_excel(f"../Relatórios/results_{timestamp}.xlsx")
predictions.to_excel(f"../Relatórios/predictions_{timestamp}.xlsx")

try:
    # Save the model to excel
    model.parameters.to_excel(f'../Relatórios/Model Summary/{Model_Name}_parameters.xlsx')
except Exception as e:
    print(f"An error occurred while saving the model parameters: {e}")


[1/1] {'RLS_option': 2, 'n_clusters': 1}

Results for NTSK-wRLS: 


RMSE: 0.6450422809739007
NRMSE: 0.2335581743594338
NDEI: 0.22889401069541743
MAE: 0.3594328388108757
MAPE: 0.1907822388548914
Rules: 1
An error occurred while saving the model parameters: Cannot save file into a non-existent directory: 'Model Summary'


### GEN-NMFIS

In [226]:
Model_Name = "GEN-NMFIS"

# Set hyperparameters range
# parameters = {'n_clusters':range(1,20)}
parameters = {'n_clusters':range(1,2)}

grid = ParameterGrid(parameters)

lower_rmse = np.inf
for i,param in enumerate(grid):
    
    print(f"[{i+1}/{len(grid)}] {param}" )

    # Optimize parameters
    model = GEN_NMFIS(**param)
    model.fit(X_train,y_train)
    # Make predictions
    y_pred = model.predict(X_val)
    
    
    # Calculating the error metrics
    # Compute the Root Mean Square Error
    RMSE = math.sqrt(mean_squared_error(y_val, y_pred))
    
    if RMSE < lower_rmse:
        lower_rmse = RMSE
        best_GEN_NMFIS_params = param

# Initialize the model
model = GEN_NMFIS(**best_GEN_NMFIS_params)
# Train the model
new_X_train, new_y_train = X.values[:index_val], y[:index_val]
OutputTraining = model.fit(new_X_train, new_y_train)
# Test the model
y_pred = model.predict(X_test)

# Save predictions to dataframe
y_pred = scaler_y.inverse_transform(y_pred.reshape(1, -1)).reshape(-1,1)
predictions[f'{Model_Name}'] = y_pred

# Model name
print(f'\nResults for {Model_Name}: \n')

# Calculating the error metrics
# Compute the Root Mean Square Error
RMSE = math.sqrt(mean_squared_error(y_test, y_pred))
print("\nRMSE:", RMSE)
# Compute the Normalized Root Mean Square Error
NRMSE = RegressionMetric(y_test, y_pred).normalized_root_mean_square_error()
print("NRMSE:", NRMSE)
# Compute the Non-Dimensional Error Index
NDEI= RMSE/st.stdev(np.asfarray(y_test.flatten()))
print("NDEI:", NDEI)
# Compute the Mean Absolute Error
MAE = mean_absolute_error(y_test, y_pred)
print("MAE:", MAE)
# Compute the Mean Absolute Percentage Error
MAPE = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE:", MAPE)
# Compute the number of final rules
Rules = model.model.parameters.shape[0]
print("Rules:", Rules)


GEN_NMFIS_ = f'{Model_Name} & {NRMSE:.2f} & {NDEI:.2f} & {MAPE:.2f} & {Rules}'

# Store results to dataframe
newrow = pd.DataFrame([[Model_Name, NRMSE, NDEI, MAPE, Rules, best_GEN_NMFIS_params]], columns=columns)
results = pd.concat([results, newrow], ignore_index=True)

# salvando em planilha
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")

results.to_excel(f"../Relatórios/results_{timestamp}.xlsx")
predictions.to_excel(f"../Relatórios/predictions_{timestamp}.xlsx")

try:
    # Save the model to excel
    model.parameters.to_excel(f'../Relatórios/Model Summary/{Model_Name}_parameters.xlsx')
except Exception as e:
    print(f"An error occurred while saving the model parameters: {e}")



[1/1] {'n_clusters': 1}


### GEN-NTSK-RLS

In [ ]:
Model_Name = "GEN-NTSK-RLS"

# Set hyperparameters range
# parameters = {'n_clusters':[1], 'lambda1':[0.95,0.96,0.97,0.98,0.99], 'RLS_option':[1]}
parameters = {'n_clusters':[1], 'lambda1':[0.95], 'RLS_option':[1]}

grid = ParameterGrid(parameters)

lower_rmse = np.inf
for i,param in enumerate(grid):
    
    print(f"[{i+1}/{len(grid)}] {param}" )

    # Optimize parameters
    model = GEN_NTSK(**param)
    model.fit(X_train,y_train)
    # Make predictions
    y_pred = model.predict(X_val)
    
    
    # Calculating the error metrics
    # Compute the Root Mean Square Error
    RMSE = math.sqrt(mean_squared_error(y_val, y_pred))
    
    if RMSE < lower_rmse:
        lower_rmse = RMSE
        best_GEN_NTSK_RLS_params = param

# Initialize the model
model = GEN_NTSK(**best_GEN_NTSK_RLS_params)
# Train the model
new_X_train, new_y_train = X.values[:index_val], y[:index_val]
OutputTraining = model.fit(new_X_train, new_y_train)
# Test the model
y_pred = model.predict(X_test)

# Save predictions to dataframe
y_pred = scaler_y.inverse_transform(y_pred.reshape(1, -1)).reshape(-1,1)
predictions[f'{Model_Name}'] = y_pred

# Model name
print(f'\nResults for {Model_Name}: \n')

# Calculating the error metrics
# Compute the Root Mean Square Error
RMSE = math.sqrt(mean_squared_error(y_test, y_pred))
print("\nRMSE:", RMSE)
# Compute the Normalized Root Mean Square Error
NRMSE = RegressionMetric(y_test, y_pred).normalized_root_mean_square_error()
print("NRMSE:", NRMSE)
# Compute the Non-Dimensional Error Index
NDEI= RMSE/st.stdev(np.asfarray(y_test.flatten()))
print("NDEI:", NDEI)
# Compute the Mean Absolute Error
MAE = mean_absolute_error(y_test, y_pred)
print("MAE:", MAE)
# Compute the Mean Absolute Percentage Error
MAPE = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE:", MAPE)
# Compute the number of final rules
Rules = model.model.parameters.shape[0]
print("Rules:", Rules)  

GEN_NTSK_RLS_ = f'{Model_Name} & {NRMSE:.2f} & {NDEI:.2f} & {MAPE:.2f} & {Rules}'

# Store results to dataframe
newrow = pd.DataFrame([[Model_Name, NRMSE, NDEI, MAPE, Rules, best_GEN_NTSK_RLS_params]], columns=columns)
results = pd.concat([results, newrow], ignore_index=True)

# salvando em planilha
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")

results.to_excel(f"../Relatórios/results_{timestamp}.xlsx")
predictions.to_excel(f"../Relatórios/predictions_{timestamp}.xlsx")

try:
    # Save the model to excel
    model.parameters.to_excel(f'../Relatórios/Model Summary/{Model_Name}_parameters.xlsx')
except Exception as e:
    print(f"An error occurred while saving the model parameters: {e}")


### GEN-NTSK-wRLS

In [ ]:
Model_Name = "GEN-NTSK-wRLS"

# Set hyperparameters range
# parameters = {'n_clusters':range(1,20), 'RLS_option':[2]}
parameters = {'n_clusters':range(1,2), 'RLS_option':[2]}

grid = ParameterGrid(parameters)

lower_rmse = np.inf
for i,param in enumerate(grid):
    
    print(f"[{i+1}/{len(grid)}] {param}" )

    # Optimize parameters
    model = GEN_NTSK(**param)
    model.fit(X_train,y_train)
    # Make predictions
    y_pred = model.predict(X_val)
    
    
    # Calculating the error metrics
    # Compute the Root Mean Square Error
    RMSE = math.sqrt(mean_squared_error(y_val, y_pred))
    
    if RMSE < lower_rmse:
        lower_rmse = RMSE
        best_GEN_NTSK_wRLS_params = param

# Initialize the model
model = GEN_NTSK(**best_GEN_NTSK_wRLS_params)
# Train the model
new_X_train, new_y_train = X.values[:index_val], y[:index_val]
OutputTraining = model.fit(X_train, y_train)
# Test the model
y_pred = model.predict(X_test)

# Save predictions to dataframe
y_pred = scaler_y.inverse_transform(y_pred.reshape(1, -1)).reshape(-1,1)
predictions[f'{Model_Name}'] = y_pred

# Model name
print(f'\nResults for {Model_Name}: \n')

# Calculating the error metrics
# Compute the Root Mean Square Error
RMSE = math.sqrt(mean_squared_error(y_test, y_pred))
print("\nRMSE:", RMSE)
# Compute the Normalized Root Mean Square Error
NRMSE = RegressionMetric(y_test, y_pred).normalized_root_mean_square_error()
print("NRMSE:", NRMSE)
# Compute the Non-Dimensional Error Index
NDEI= RMSE/st.stdev(np.asfarray(y_test.flatten()))
print("NDEI:", NDEI)
# Compute the Mean Absolute Error
MAE = mean_absolute_error(y_test, y_pred)
print("MAE:", MAE)
# Compute the Mean Absolute Percentage Error
MAPE = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE:", MAPE)
# Compute the number of final rules
Rules = model.model.parameters.shape[0]
print("Rules:", Rules)  


GEN_NTSK_wRLS_ = f'{Model_Name} & {NRMSE:.2f} & {NDEI:.2f} & {MAPE:.2f} & {Rules}'

# Store results to dataframe
newrow = pd.DataFrame([[Model_Name, NRMSE, NDEI, MAPE, Rules, best_GEN_NTSK_wRLS_params]], columns=columns)
results = pd.concat([results, newrow], ignore_index=True)

# salvando em planilha
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")

results.to_excel(f"../Relatórios/results_{timestamp}.xlsx")
predictions.to_excel(f"../Relatórios/predictions_{timestamp}.xlsx")

try:
    # Save the model to excel
    model.parameters.to_excel(f'../Relatórios/Model Summary/{Model_Name}_parameters.xlsx')
except Exception as e:
    print(f"An error occurred while saving the model parameters: {e}")
